# 🚦 NYC Traffic Analytics Notebook
## Blade Runner Surveillance System - Traffic Intelligence

Comprehensive analysis of NYC traffic data including:
- **Traffic Events** (incidents, construction, closures)
- **Traffic Speeds** (real-time congestion analysis)
- **Camera Surveillance** (coverage and status)

Data streamed via **Snowpipe Streaming High Speed v2 REST API** from 511NY.

In [ ]:
-- Set context
USE DATABASE DEMO;
USE SCHEMA DEMO;
USE WAREHOUSE INGEST;

---
## 📊 Data Overview

In [ ]:
-- Data volume summary across all tables
SELECT 
    'NYC_CAMERA_DATA' as table_name,
    COUNT(*) as total_rows,
    COUNT(DISTINCT camera_id) as unique_entities,
    MIN(image_timestamp) as first_record,
    MAX(image_timestamp) as last_record
FROM NYC_CAMERA_DATA
UNION ALL
SELECT 
    'NYC_TRAFFIC_EVENTS',
    COUNT(*),
    COUNT(DISTINCT event_id),
    MIN(event_timestamp),
    MAX(event_timestamp)
FROM NYC_TRAFFIC_EVENTS
UNION ALL
SELECT 
    'NYC_TRAFFIC_SPEEDS',
    COUNT(*),
    COUNT(DISTINCT segment_id),
    MIN(traffic_timestamp),
    MAX(traffic_timestamp)
FROM NYC_TRAFFIC_SPEEDS
ORDER BY table_name;

---
## 🚨 Traffic Events Analysis

In [ ]:
-- Event type distribution
SELECT 
    event_type,
    COUNT(*) as event_count,
    COUNT(DISTINCT event_id) as unique_events,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM NYC_TRAFFIC_EVENTS
GROUP BY event_type
ORDER BY event_count DESC;

In [ ]:
-- Top roadways affected by events
SELECT 
    roadway_name,
    COUNT(*) as total_events,
    COUNT(DISTINCT event_type) as event_types,
    LISTAGG(DISTINCT event_type, ', ') WITHIN GROUP (ORDER BY event_type) as types
FROM NYC_TRAFFIC_EVENTS
WHERE roadway_name IS NOT NULL AND roadway_name != ''
GROUP BY roadway_name
ORDER BY total_events DESC
LIMIT 15;

---
## 🏎️ Traffic Speed Analysis

In [ ]:
-- Overall speed statistics
SELECT 
    COUNT(*) as total_readings,
    COUNT(DISTINCT segment_id) as unique_segments,
    ROUND(AVG(current_speed), 2) as avg_current_speed,
    ROUND(AVG(free_flow_speed), 2) as avg_free_flow_speed,
    ROUND(AVG(CASE WHEN free_flow_speed > 0 
              THEN current_speed / free_flow_speed ELSE NULL END) * 100, 2) as avg_flow_percentage,
    ROUND(MIN(current_speed), 2) as min_speed,
    ROUND(MAX(current_speed), 2) as max_speed
FROM NYC_TRAFFIC_SPEEDS
WHERE current_speed > 0;

In [ ]:
-- Congestion hotspots (slowest segments)
SELECT 
    segment_id,
    roadway_name,
    direction,
    from_location,
    to_location,
    ROUND(AVG(current_speed), 1) as avg_speed,
    ROUND(AVG(free_flow_speed), 1) as free_flow,
    ROUND(AVG(CASE WHEN free_flow_speed > 0 
              THEN current_speed / free_flow_speed ELSE NULL END) * 100, 1) as flow_pct,
    CASE 
        WHEN AVG(CASE WHEN free_flow_speed > 0 
                 THEN current_speed / free_flow_speed ELSE NULL END) >= 0.8 THEN '🟢 Good'
        WHEN AVG(CASE WHEN free_flow_speed > 0 
                 THEN current_speed / free_flow_speed ELSE NULL END) >= 0.5 THEN '🟡 Moderate'
        ELSE '🔴 Congested'
    END as status,
    COUNT(*) as reading_count
FROM NYC_TRAFFIC_SPEEDS
WHERE traffic_timestamp >= DATEADD('hour', -6, CURRENT_TIMESTAMP())
  AND current_speed > 0
GROUP BY segment_id, roadway_name, direction, from_location, to_location
HAVING COUNT(*) >= 2
ORDER BY flow_pct ASC
LIMIT 20;

In [ ]:
-- Speed distribution by roadway with traffic status
SELECT 
    roadway_name,
    COUNT(DISTINCT segment_id) as segments,
    ROUND(AVG(current_speed), 1) as avg_speed,
    ROUND(AVG(free_flow_speed), 1) as avg_free_flow,
    ROUND(AVG(CASE WHEN free_flow_speed > 0 
              THEN current_speed / free_flow_speed ELSE NULL END) * 100, 1) as flow_pct,
    CASE 
        WHEN AVG(CASE WHEN free_flow_speed > 0 
                 THEN current_speed / free_flow_speed ELSE NULL END) >= 0.8 THEN '🟢 Good'
        WHEN AVG(CASE WHEN free_flow_speed > 0 
                 THEN current_speed / free_flow_speed ELSE NULL END) >= 0.5 THEN '🟡 Moderate'
        ELSE '🔴 Congested'
    END as traffic_status
FROM NYC_TRAFFIC_SPEEDS
WHERE roadway_name IS NOT NULL AND roadway_name != ''
GROUP BY roadway_name
HAVING COUNT(*) >= 10
ORDER BY flow_pct ASC
LIMIT 25;

---
## 📷 Camera Coverage Analysis

In [ ]:
-- Camera coverage by roadway
SELECT 
    roadway_name,
    COUNT(DISTINCT camera_id) as cameras,
    COUNT(*) as total_captures,
    ROUND(AVG(latitude), 4) as center_lat,
    ROUND(AVG(longitude), 4) as center_lon
FROM NYC_CAMERA_DATA
WHERE roadway_name IS NOT NULL AND roadway_name != ''
GROUP BY roadway_name
ORDER BY cameras DESC
LIMIT 20;

---
## 📈 Semantic View Queries
Using SQL semantic views for standardized analytics.

In [ ]:
-- Query traffic speeds semantic view - congestion analysis
SELECT * FROM SEMANTIC_VIEW(
  NYC_TRAFFIC_SPEEDS_SEMANTIC_VIEW
  DIMENSIONS speeds.roadway, speeds.direction
  METRICS speeds.avg_current_speed, speeds.avg_congestion, speeds.total_readings
) ORDER BY avg_congestion ASC LIMIT 20;

In [ ]:
-- Query traffic events semantic view
SELECT * FROM SEMANTIC_VIEW(
  NYC_TRAFFIC_EVENTS_SEMANTIC_VIEW
  DIMENSIONS events.event_type, events.roadway
  METRICS events.total_events, events.unique_events
) ORDER BY total_events DESC;

---
## 📝 Summary

This notebook provides comprehensive traffic analytics for NYC using data streamed via Snowpipe Streaming High Speed v2.

*"I've seen things you people wouldn't believe... All those moments will be lost in time, like tears in rain."* - Blade Runner